In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np

from PIL import Image,ImageDraw

import tensorflow as tf # TF2


def load_labels(filename):
  with open(filename, 'r') as f:
    return [line.strip() for line in f.readlines()]


if __name__ == '__main__':
    face_model = "./face_landmark.tflite"
    iris_model = "./iris_landmark.tflite"
    landmark_file = "./landmark_contours.txt"

    interpreter = tf.lite.Interpreter(model_path=face_model)
    interpreter.allocate_tensors()

    iris_interpreter = tf.lite.Interpreter(model_path=iris_model)
    iris_interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    print("face landmark input info:",input_details)
    output_details = interpreter.get_output_details()

    iris_input = iris_interpreter.get_input_details()
    print("iris landmark input info:",iris_input)
    iris_output = iris_interpreter.get_output_details()

    # check the type of the input tensor
    floating_model = input_details[0]['dtype'] == np.float32
    iris_float_model = iris_input[0]["dtype"] == np.float32

    # NxHxWxC, H:1, W:2
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]

    iris_height = iris_input[0]["shape"][1]
    iris_width = iris_input[0]["shape"][2]

    print("height = ",height,"width = ",width)
    img = Image.open("./img/face.jpeg").resize((width, height))
    draw = ImageDraw.Draw(img)
    print("numpy img size = ",img.size)

    eye_points = open(landmark_file,"r")
    left_eye_points = []
    right_eye_points = []
    for line in eye_points:
        points = line.split(" ")
        if points[0] == "left_eye":
            for data in points[1:-1]:
                left_eye_points.append(int(data))
        elif points[0] == "right_eye":
            for data in points[1:-1]:
                right_eye_points.append(int(data))
    print("left eye points ",left_eye_points)
    print("right eye points ",right_eye_points)

    # add N dim
    input_data = np.expand_dims(img, axis=0)

    if floating_model:
        input_data = (np.float32(input_data) - 128) / 256 

    interpreter.set_tensor(input_details[0]['index'], input_data)

    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])
    results = np.squeeze(output_data)

    output_data1= interpreter.get_tensor(output_details[1]["index"])
    print("output data 1 = ",output_data1)

    print ("result size = ",len(results))
    output_file = open("point_file.txt","w")
    point_list = []
    color_list = [0,0,0]
    left_eye_list = [[],[]]
    right_eye_list = [[],[]]
    left_eye_center = [0,0]
    right_eye_center = [0,0]
    for index in range(int(len(results)/3)):
        point = (results[3*index],results[3*index + 1])
        if index in left_eye_points:
            draw.point(point,(255,0,0))
            print(index,point,"\n")
            left_eye_list[0].append(point[0])
            left_eye_list[1].append(point[1])
        elif index in right_eye_points:
            draw.point(point,(255,0,0))
            right_eye_list[0].append(point[0])
            right_eye_list[1].append(point[1])
        else :
            draw.point(point,(0,255,0))
        # draw.point(point,tuple( color_list))
        point_list.append(point)
        output_file.writelines(str(point))
        output_file.write("\n")

    print("left eye center count",left_eye_list)
    left_eye_center[1] = sum(left_eye_list[1])/len(left_eye_list[1])
    left_eye_center[0] = sum(left_eye_list[0])/len(left_eye_list[0])

    right_eye_center[1] = sum(right_eye_list[1])/len(right_eye_list[1])
    right_eye_center[0] = sum(right_eye_list[0])/len(right_eye_list[0])
    print("left eye center",left_eye_center)
    draw.point(left_eye_center,(0,0,255))
    draw.point(right_eye_center,(0,0,255))
    # draw.point((71.96,91.39),(0,0,255))
    # draw.point(point_list,(0,255,0))

    left_eye_box = (left_eye_center[0]-32,left_eye_center[1]-32,left_eye_center[0]+32,left_eye_center[1]+32)
    right_eye_box = (right_eye_center[0]-32,right_eye_center[1]-32,right_eye_center[0]+32,right_eye_center[1]+32)
    iris_left_img = Image.open("./img/face.jpeg").resize((width, height)).crop(left_eye_box)
    iris_right_img = Image.open("./img/face.jpeg").resize((width, height)).crop(right_eye_box)
    img.show("sources img")

    # add N dim
    iris_input_data = np.expand_dims(iris_left_img, axis=0)

    if iris_float_model:
        iris_input_data = (np.float32(iris_input_data) - 128) / 256 

    iris_interpreter.set_tensor(iris_input[0]['index'], iris_input_data)

    iris_interpreter.invoke()

    iris_output_data = iris_interpreter.get_tensor(iris_output[0]['index'])
    iris_results = np.squeeze(iris_output_data)

    iris_output_data1= iris_interpreter.get_tensor(iris_output[1]["index"])
    print("iris data 1 = ",iris_output_data1)
    iris_results = np.squeeze(iris_output_data1)
    
    iris_draw = ImageDraw.Draw(iris_left_img)
    for index in range(int(len(iris_results)/3)):
        point = (iris_results[3*index],iris_results[3*index + 1])
        iris_draw.point(point,(0,0,255))
    iris_left_img.show("left eye img")



face landmark input info: [{'name': 'input_1', 'index': 0, 'shape': array([  1, 192, 192,   3], dtype=int32), 'shape_signature': array([  1, 192, 192,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
iris landmark input info: [{'name': 'input_1', 'index': 0, 'shape': array([ 1, 64, 64,  3], dtype=int32), 'shape_signature': array([ 1, 64, 64,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
height =  192 width =  192
numpy img size =  (192, 192)
left eye points  [33, 7, 7, 163, 163, 144, 144, 145, 145, 153, 153, 154, 154, 155, 155, 133, 33, 246, 246, 161, 161, 160, 160, 159, 159, 158, 158, 157, 157, 173, 173]
right e